In [1]:
using Pkg
Pkg.activate(".")

using DataFrames
using NemoMod
using SQLite
using Cbc
using JuMP


include("setup_analysis.jl")

 Activating environment at `~/Documents/Projects/git_jbus/lac_decarbonization/julia/Project.toml`


"/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/ref"

In [9]:
include("setup_analysis.jl")

"/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/ref"

In [20]:
#
#  TEMPORARY LOCATIONS FOR PROOF OF CONCEPT
#

dir_attributes = "/Users/jsyme/Documents/Projects/git_jbus/lac-tutorial/docs/source/csvs"

at_fuel = AttributeTable(
    joinpath(dir_attributes, "attribute_cat_fuel.csv"),
    Symbol("``\$CAT-FUEL\$``"), 
    [Symbol("Category Name")],
    true
);

at_region = AttributeTable(
    joinpath(dir_attributes, "attribute_cat_region.csv"),
    Symbol("``\$REGION\$``"),
    [Symbol("Category Name")],
    true
);

at_technology = AttributeTable(
    joinpath(dir_attributes, "attribute_cat_technology.csv"),
    Symbol("``\$CAT-TECHNOLOGY\$``"),
    [Symbol("Category Name")],
    true
);




┌ Warning: thread = 1 warning: only found 5 / 6 columns around data row: 12. Filling remaining columns with `missing`
└ @ CSV /Users/jsyme/.julia/packages/CSV/Zl2ww/src/file.jl:612
┌ Warning: thread = 1 warning: only found 5 / 6 columns around data row: 23. Filling remaining columns with `missing`
└ @ CSV /Users/jsyme/.julia/packages/CSV/Zl2ww/src/file.jl:612
┌ Warning: thread = 1 warning: only found 5 / 6 columns around data row: 11. Filling remaining columns with `missing`
└ @ CSV /Users/jsyme/.julia/packages/CSV/Zl2ww/src/file.jl:612


In [21]:
at_fuel.key_vals

11-element Array{String,1}:
 "fuel_biomass"
 "fuel_coal"
 "fuel_electricity"
 "fuel_gas"
 "fuel_geothermal"
 "fuel_hydropower"
 "fuel_nuclear"
 "fuel_oil"
 "fuel_other"
 "fuel_solar"
 "fuel_wind"

In [16]:
# get technology categories for integration into tables
keys_technology = collect(at_technology.table[:, at_technology.key])
categories_technology = [replace(at_technology.field_maps["cat_technology_to_category_name"][x], "`" => "") for x in keys_technology]
categories_technology

##  
keys_sorted = sort(collect(keys(dict_tables)))
files_to_specify = []
for k in keys_sorted
    if nrow(dict_tables[k]) > 0
        push!(files_to_specify, k)
    end
end;


LoadError: [91mUndefVarError: dict_tables not defined[39m

In [17]:

##  function for getting powerplants
function get_powerplants(
        fp_powerplant_data::String, 
        dict_ext::Dict{Symbol, String}, 
        attribute_tech::AttributeTable,
        attribute_region::AttributeTable,
        field_region::Symbol = :country,
        field_fuel::Symbol = :primary_fuel
    )
    
    # checkt the dictionary
    if !issubset(Set(values(dict_ext)), Set(["group", "sum", "mean"]))
        error("Error in dict_ext: invalid values.")
    end
    fields_ext = Symbol.(collect(keys(dict_ext)))
    fields_grp = [Symbol(x) for x in keys(dict_ext) if (dict_ext[x] == "group")]
    dict_agg = Dict()
    for k in fields_ext
        if !(k in fields_grp)
            if dict_ext[k] == "sum"
                dict_agg[k] = sum
            elseif dict_agg[k] == "mean"
                dict_ext[k] = mean
            end
        end
    end
    fields_data = Symbol.(collect(keys(dict_agg)))
    
    
    # get the dataframe
    fp_powerplant_data = check_path(fp_powerplant_data, false)
    df_pp = read_csv(fp_powerplant_data, true)
    # check fields in the extraction dictionary 
    check_fields!(df_pp, fields_ext)
    
    
    ##  setup techs to replace (from attribute table)
    categories_tech = Set(replace.(attribute_tech.table[:, "cat_technology"], "`" => ""))
    set_fuels_pp = union([Set(df_pp[:, x]) for x in names(df_pp) if occursin("fuel", x)]...);
    tech_src = collect(skipmissing(set_fuels_pp))
    tech_targ = "pp_".*replace(lowercase.(tech_src), "hydro" => "hydropower")
    for i in 1:length(tech_src)
        tech = tech_targ[i]
        if !(tech in categories_tech)
            tech_targ[i] = "pp_other"
        end
    end
    # a dictionary to replace the values with the techs in
    dict_pp_repl = Dict(zip(tech_src, tech_targ))
    all_pp_for_cross = sort(collect(intersect(categories_tech, Set(tech_targ))))

    ##  get region info (lower case)
    all_regions = replace.(keys(at_region.field_maps["region_to_category_name"]), "`" => "")

    ##  very basic approach -- get energy totals by country
    df_energy = copy(df_pp[:, fields_ext])
    for nm in fields_grp
        df_energy[!, nm] = String.(df_energy[!, nm])
    end 
    for fld in Symbol.(collect(keys(dict_agg)))
        df_energy[!, fld] = Float64.(df_energy[!, fld])
    end
    
    df_energy[!, field_fuel] = replace(df_energy[:, field_fuel], dict_pp_repl...)
    df_energy[!, field_region] = lowercase.(df_energy[:, field_region])
    df_energy = combine(groupby(df_energy, fields_grp), Dict(:capacity_mw => sum)..., renamecols = false);
    df_energy0 = copy(df_energy)
    #reshape(collect(Iterators.product([1, 2, 3], [1, 2, 3], [1, 2])), (18, ))
    dfe = crossjoin(DataFrame(field_region => all_regions), DataFrame(field_fuel => all_pp_for_cross))
    df_energy = leftjoin(dfe, df_energy, on = Symbol.(names(dfe)));
    for fld in fields_data
        df_energy[!, fld] = replace(coalesce(df_energy[!, fld]), missing => 0.0);
    end
    
    return sort(df_energy, [field_region, field_fuel])
end


df_energy = get_powerplants(
    "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/LAC_global_power_plant_database.csv",
    Dict(:country => "group", :primary_fuel => "group", :capacity_mw => "sum"),
    at_technology,
    at_region
);




In [47]:
##########################################################################
###                                                                    ###
###    DEFINE FUNCTIONS TO CONVERT DATA TO NEMO MOD INPUT DB TABLES    ###
###                                                                    ###
##########################################################################

# attribute table
function build_nemo_attribute(attribute_table::AttributeTable, field_category::Symbol, field_description::Symbol)
    
    df_out = copy(attribute_table.table)
    fields_ext = [field_category, field_description]
    check_fields!(df_out, fields_ext)
    df_out = df_out[:, [field_category, field_description]]
    
    rename!(df_out, Dict(field_category => :val, field_description => :desc))
    
    return df_out
end




build_nemo_attribute (generic function with 3 methods)

In [53]:
#######################################
#    build tables to export to SQL    #
#######################################

dict_tables_out = Dict{String, DataFrame}()

dict_tables_out["FUEL"] = build_nemo_attribute(at_fuel, at_fuel.key, :description)
dict_tables_out["TECHNOLOGY"] = build_nemo_attribute(at_technology, at_technology.key, :description)
dict_tables_out["REGION"] = build_nemo_attribute(at_region, at_region.key, :category_name)



,val,desc
,String,String
1,arg,Argentina
2,bol,Bolivia
3,bra,Brazil
4,chl,Chile
5,col,Colombia
6,cri,Costa Rica
7,dom,Dominican Republic
8,ecu,Ecuador
9,slv,El Salvador


In [ ]:
##### reshape(collect(Iterators.product([1, 2, 3], [1, 2, 3], [1, 2])), (18, ))

In [46]:
for k in keys(dict_tables)
    if issubset(Set(["val", "desc"]), Set(names(dict_tables[k])))
        print("$(k)\n")
    end
end

EMISSION
FUEL
STORAGE
REGION
TECHNOLOGY
NODE
MODE_OF_OPERATION
TIMESLICE
YEAR


In [ ]:
#### dbpath = normpath(joinpath(pathof(NemoMod), "..", "..", "test", "storage_test.sqlite"))
# it's important to set numprocs = 1; the parallelization doesn't seem to export the package (or the environment?). need to report
NemoMod.calculatescenario(dbpath; jumpmodel=Model(Cbc.Optimizer), numprocs = 1)



In [26]:
dbpath = normpath(joinpath(pathof(NemoMod), "..", "..", "test", "storage_test.sqlite"))
db = SQLite.DB(dbpath)

SQLite.DB("/Users/jsyme/.julia/packages/NemoMod/OkhEV/test/storage_test.sqlite")

LoadError: [91mUndefVarError: dbpath not defined[39m

In [27]:
#SQLite.columns(db, "EMISSION")

dict_tables = Dict{String, DataFrame}()
tables_vary = []

for k in SQLite.tables(db)[1]
    df_tmp = DBInterface.execute(db, "select * from $(k)") |> DataFrame
    dict_tables[k] = df_tmp
    
    if nrow(df_tmp) > 0
        push!(tables_vary, k)
    end
    
    fp_out = joinpath(dir_out, "$(k).csv")
    
    if !ispath(fp_out)
        CSV.write(fp_out, df_tmp)
    end
    
    print("\nFinished with table $(k)...\n")
end;



Finished with table EMISSION...

Finished with table FUEL...

Finished with table MODE_OF_OPERATION...

Finished with table REGION...

Finished with table TECHNOLOGY...

Finished with table TIMESLICE...

Finished with table TSGROUP1...

Finished with table TSGROUP2...

Finished with table YEAR...

Finished with table YearSplit...

Finished with table VariableCost...

Finished with table TradeRoute...

Finished with table TotalTechnologyModelPeriodActivityUpperLimit...

Finished with table TotalTechnologyModelPeriodActivityLowerLimit...

Finished with table TotalTechnologyAnnualActivityUpperLimit...

Finished with table TotalTechnologyAnnualActivityLowerLimit...

Finished with table TotalAnnualMinCapacityInvestment...

Finished with table TotalAnnualMinCapacityInvestmentStorage...

Finished with table TotalAnnualMinCapacity...

Finished with table TotalAnnualMinCapacityStorage...

Finished with table TotalAnnualMaxCapacityInvestment...

Finished with table TotalAnnualMaxCapacityInvestm

In [49]:
DBInterface.execute(db, "select * from REGION") |> DataFrame

,val,desc
,String,Missing
1,1,missing


In [33]:
[x for x in keys(dict_tables) if "$(x[1])" == "v"]

8-element Array{String,1}:
 "vusenn"
 "vdemandnn"
 "vproductionnn"
 "vtotaldiscountedcost"
 "vusebytechnologyannual"
 "vproductionbytechnologyannual"
 "vnewcapacity"
 "vtotalcapacityannual"

In [32]:
for k in sort(collect(keys(dict_tables)))
    print("\n$(k)")
end


AccumulatedAnnualDemand
AnnualEmissionLimit
AnnualExogenousEmission
AvailabilityFactor
CapacityFactor
CapacityOfOneTechnologyUnit
CapacityToActivityUnit
CapitalCost
CapitalCostStorage
DefaultParams
DepreciationMethod
DiscountRate
EMISSION
EmissionActivityRatio
EmissionsPenalty
FUEL
FixedCost
InputActivityRatio
InterestRateStorage
InterestRateTechnology
LTsGroup
MODE_OF_OPERATION
MinStorageCharge
MinimumUtilization
ModelPeriodEmissionLimit
ModelPeriodExogenousEmission
NODE
NodalDistributionDemand
NodalDistributionStorageCapacity
NodalDistributionTechnologyCapacity
OperationalLife
OperationalLifeStorage
OutputActivityRatio
REGION
REMinProductionTarget
RETagFuel
RETagTechnology
RampRate
RampingReset
ReserveMargin
ReserveMarginTagFuel
ReserveMarginTagTechnology
ResidualCapacity
ResidualStorageCapacity
STORAGE
SpecifiedAnnualDemand
SpecifiedDemandProfile
StorageFullLoadHours
StorageLevelStart
StorageMaxChargeRate
StorageMaxDischargeRate
TECHNOLOGY
TIMESLICE
TSGROUP1
TSGROUP2
TechnologyFrom